In [1]:
# ! pip install ddaclient>=1.1.6
import ddaclient

ddac = ddaclient.RemoteDDA(service_url="private=https://dda-interface-private.obsuks1.unige.ch,public=http://dda-interface-public.obsuks1.unige.ch", 
                           ddcache_root_local="/data/reduced/ddcache",
                        #    ddcache_root_local="/mnt/sshfs/lesta/projects/astro/integral/data/reduced/ddcache-public-usr_astro_in-p1"
                           )

In [2]:
from odafunction.executors import default_execute_to_value
from odafunction.func.urifunc import URIipynbFunction, URIValue


v = default_execute_to_value(URIipynbFunction.from_generic_uri("file:///home/savchenk/grb221009/grbbase.ipynb")(focus_obs_name="total_3p"), valueclass=URIValue, cached=True)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:root:big problem with sentry: FileNotFoundError(2, 'No such file or directory')
DEBUG:matplotlib:matplotlib data path: /home/savchenk/miniconda-recent/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/savchenk/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/savchenk/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/savchenk/.cache/matplotlib/fontlist-v330.json
INFO:odafunction.func.urifunc:constructing <class 'odafunction.func.urifunc.URIipynbFunction'> from uri=file:///home/savchenk/grb221009/grbbase.ipynb value=None provenance=None
INFO:odafunction.func.urifunc:parsing URI file:///home/savchenk/grb22100...(45)
INFO:odafunction.func.urifunc:parsed uri file:///home/savchenk/grb221009/grbbase.ipynb as {'modifier': None, 'schema': 'file', 'path': '/home/savchenk/grb221009/grbbase.ipynb', 'f

In [3]:
import pandas as pd
import requests
from astropy.coordinates import SkyCoord
from astropy import units as u

src_coord = SkyCoord("19h 13m 03.43s", "+19d 46m 23.1s", unit=u.deg)

scw_dict_all = pd.DataFrame(requests.get(f"https://www.astro.unige.ch/mmoda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/2022-10-09T00:00:00/2023-10-23T00:00:00?"
             f"ra={src_coord.ra.deg}&dec={src_coord.dec.deg}&radius=15&return_columns=SWID,SW_TYPE,TELAPSE,TSTART,TSTOP").json())


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.astro.unige.ch:443
DEBUG:urllib3.connectionpool:https://www.astro.unige.ch:443 "GET /mmoda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/2022-10-09T00:00:00/2023-10-23T00:00:00?ra=288.26429166666657&dec=19.773083333333332&radius=15&return_columns=SWID,SW_TYPE,TELAPSE,TSTART,TSTOP HTTP/1.1" 200 28680


In [4]:
cat = ["SourceCatalog", {"autoversion": True, "catalog": [
            {"NAME":"SGR 1935+2154", "RA": 293.73200 , "DEC":  21.89672},
            {"NAME":"GRB 221009A", "RA": src_coord.ra.deg , "DEC":  src_coord.dec.deg}
]}]

In [5]:
rs = []

for scw in scw_dict_all[scw_dict_all.SW_TYPE.str.strip()=='POINTING'].SWID:
    print(scw)

    try:
        r = ddac.query(target="ii_light", 
                prompt_delegate=True,
                modules=["git://ddosa/staging-1-3", "git://findic/staging-1-3-icversion", "git://ddosa11/staging-1-3", "git://integral_all_private", "git://ii_light", "git://gencat/staging-1-3"],
                assume=[
                                f'ddosa.ScWData(input_scwid="{scw}.001")',
                                'ii_light.TimeBin(use_tbin=1)'
                        ],
                inject=[cat])
        rs.append(r)
    except ddaclient.AnalysisDelegatedException:
        pass
#     break

INFO:ddaclient:(('modules', 'git://ddosa/staging-1-3,git://findic/staging-1-3-icversion,git://ddosa11/staging-1-3,git://integral_all_private,git://ii_light,git://gencat/staging-1-3'), {})
INFO:ddaclient:(('assume', 'ddosa.ScWData(input_scwid="255800700010.001"),ii_light.TimeBin(use_tbin=1)'), {})
INFO:ddaclient:(('service url:', 'https://dda-interface-private.obsuks1.unige.ch'), {})
INFO:ddaclient:(('target:', 'ii_light'), {})
INFO:ddaclient:(('inject:', [['SourceCatalog', {'autoversion': True, 'catalog': [{'NAME': 'SGR 1935+2154', 'RA': 293.732, 'DEC': 21.89672}, {'NAME': 'GRB 221009A', 'RA': 288.26429166666657, 'DEC': 19.773083333333332}]}]]), {})
INFO:ddaclient:request to pipeline with parameters: {'url': 'https://dda-interface-private.obsuks1.unige.ch/api/v2.0/ii_light', 'params': {'modules': 'git://ddosa,git://ddosa/staging-1-3,git://findic/staging-1-3-icversion,git://ddosa11/staging-1-3,git://integral_all_private,git://ii_light,git://gencat/staging-1-3', 'assume': 'ddosa.ScWData(

255800700010


DEBUG:urllib3.connectionpool:https://dda-interface-private.obsuks1.unige.ch:443 "POST /api/v2.0/ii_light HTTP/1.1" 200 16984
DEBUG:ddaclient:{"assume":"ddosa.ScWData(input_scwid=\"255800700010.001\"),ii_light.TimeBin(use_tbin=1)","cached_path":["/data/reduced/ddcache/byscw/2558/255800700010.001/ii_light.v0//cce9a91b/"],"data":{"_da_stored_link_input_binning":"TimeBin","_da_stored_link_input_bins":"ImageBins","_da_stored_link_input_dead":"ibis_dead","_da_stored_link_input_events":"ISGRIEvents","_da_stored_link_input_gti":"ibis_gti","_da_stored_link_input_maps":"BinMapsSpectra","_da_stored_link_input_scw":"ScWData","_da_stored_link_input_spectra":"ii_spectra_extract","abstract":false,"allow_alias":false,"allow_timespent_adjustment":false,"copy_cached_input":true,"datafile_restore_mode":"None","default_log_level":"","essential_deep_inputs":[],"explicit_output":"None","force_complete_input":true,"hard_timespent_checks":false,"incomplete":false,"infactory":true,"lc":["DataFile","lcr.fits","

255800710010


DEBUG:urllib3.connectionpool:https://dda-interface-private.obsuks1.unige.ch:443 "POST /api/v2.0/ii_light HTTP/1.1" 200 16990
DEBUG:ddaclient:{"assume":"ddosa.ScWData(input_scwid=\"255800710010.001\"),ii_light.TimeBin(use_tbin=1)","cached_path":["/data/reduced/ddcache/byscw/2558/255800710010.001/ii_light.v0//cce9a91b/"],"data":{"_da_stored_link_input_binning":"TimeBin","_da_stored_link_input_bins":"ImageBins","_da_stored_link_input_dead":"ibis_dead","_da_stored_link_input_events":"ISGRIEvents","_da_stored_link_input_gti":"ibis_gti","_da_stored_link_input_maps":"BinMapsSpectra","_da_stored_link_input_scw":"ScWData","_da_stored_link_input_spectra":"ii_spectra_extract","abstract":false,"allow_alias":false,"allow_timespent_adjustment":false,"copy_cached_input":true,"datafile_restore_mode":"None","default_log_level":"","essential_deep_inputs":[],"explicit_output":"None","force_complete_input":true,"hard_timespent_checks":false,"incomplete":false,"infactory":true,"lc":["DataFile","lcr.fits","

...

255800820020


DEBUG:urllib3.connectionpool:https://dda-interface-private.obsuks1.unige.ch:443 "POST /api/v2.0/ii_light HTTP/1.1" 200 16335
DEBUG:ddaclient:{"assume":"ddosa.ScWData(input_scwid=\"255800820020.001\"),ii_light.TimeBin(use_tbin=1)","cached_path":["/data/reduced/ddcache/byscw/2558/255800820020.001/ii_light.v0//cce9a91b/"],"data":{"_da_stored_link_input_binning":"TimeBin","_da_stored_link_input_bins":"ImageBins","_da_stored_link_input_dead":"ibis_dead","_da_stored_link_input_events":"ISGRIEvents","_da_stored_link_input_gti":"ibis_gti","_da_stored_link_input_maps":"BinMapsSpectra","_da_stored_link_input_scw":"ScWData","_da_stored_link_input_spectra":"ii_spectra_extract","abstract":false,"allow_alias":false,"allow_timespent_adjustment":false,"copy_cached_input":true,"datafile_restore_mode":"None","default_log_level":"","essential_deep_inputs":[],"explicit_output":"None","force_complete_input":true,"hard_timespent_checks":false,"incomplete":false,"infactory":true,"lc":["DataFile","lcr.fits","

255900010010


DEBUG:urllib3.connectionpool:https://dda-interface-private.obsuks1.unige.ch:443 "POST /api/v2.0/ii_light HTTP/1.1" 200 17571
DEBUG:ddaclient:{"assume":"ddosa.ScWData(input_scwid=\"255900010010.001\"),ii_light.TimeBin(use_tbin=1)","cached_path":"unreable","data":"unreadable-object-data","exceptions":{"exception":["dataanalysis.core.AnalysisException","{}"],"exception_type":"node"},"hashe":"unreable","modules":["git://ddosa","git://ddosa/staging-1-3","git://findic/staging-1-3-icversion","git://ddosa11/staging-1-3","git://integral_all_private","git://ii_light","git://gencat/staging-1-3"],"result":"INFO:from_uri:found ODAHUB URI option: https://crux-private.obsuks1.unige.ch@default\nINFO:[ QueueProxy: leader=https://crux-private.obsuks1.unige.ch queue=default ]:initialized\nINFO:from_uri:probing connection...\nINFO:from_uri:succeeded!\ninitialized dqueue: [ QueueProxy: leader=https://crux-private.obsuks1.unige.ch queue=default ]\n1676649560.095118 \u001btop\u001b 20263/140092617656128 will

AnalysisException: ('found analysis exceptions', [['dataanalysis.core.AnalysisException', '{}']])

In [ ]:
from astropy.time import Time

T0 = Time(v['output_values']['data']['T0_isot'])
T0_ijd = T0.mjd - 51544

In [ ]:
from astropy.io import fits
from matplotlib import pylab as plt


d = fits.open(r.lc)[2].data

plt.plot(d['TIME'] - T0_ijd, d['RATE'])